In [105]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# import nltk
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
# import CountVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB #remove
from sklearn.metrics import confusion_matrix, accuracy_score


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ritik\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [106]:
import os
for dirname, _, filenames in os.walk('/dataset'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [107]:
data = pd.read_csv('dataset/complaints.csv')

C:\Users\ritik\AppData\Local\Temp\ipykernel_4636\3970970897.py:1: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('dataset/complaints.csv')


In [114]:
data.head()

,Date_Received,Product,Sub_Product,Issue,Sub_Issue,Complaint,Response,Company,Submitted_Via,Date_Sent_To_Company,Company_Response_To_Consumer,Timely_Response?,Consumer_Disputed?,Complaint_Id
114848,2016-01-18,Debt collection,Credit card,Cont'd attempts collect debt not owed,Debt was discharged in bankruptcy,I was issued a credit due to me by XXXX the re...,Company chooses not to provide a public response,"CITIBANK, N.A.",Web,2016-01-18,Closed with explanation,Yes,No,1746591
114870,2017-02-23,Debt collection,Auto,Disclosure verification of debt,Not given enough info to verify debt,I XXXX XXXX file this notice with the Consumer...,Company has responded to the consumer and the ...,CREDIT ACCEPTANCE CORPORATION,Web,2017-02-23,Closed with explanation,Yes,No,2356567
114941,2017-01-20,Debt collection,I do not know,Cont'd attempts collect debt not owed,Debt is not mine,"Received a phone call from XXXX, WA ( XXXX ), ...",Company believes complaint is the result of an...,"GMA Investments, LLC",Web,2017-03-24,Closed with explanation,Yes,No,2298566
114982,2017-02-21,Debt collection,I do not know,Disclosure verification of debt,Not given enough info to verify debt,Syndicated Office Systems is violating Federal...,Company believes it acted appropriately as aut...,TENET HEALTHCARE CORPORATION,Web,2017-02-21,Closed with explanation,Yes,No,2353346
114995,2017-04-05,Debt collection,Auto,Improper contact or sharing of info,Talked to a third party about my debt,Ally Bank Collections called a distant relativ...,Company has responded to the consumer and the ...,ALLY FINANCIAL INC.,Web,2017-04-05,Closed with explanation,Yes,No,2421685


In [110]:
print("Number of dimensions in dataset : ", data.ndim)
print("Dimensions of dataset : ", data.shape)
print("Number of Features in the dataset: ", len(data.columns))
print("List of features in the dataset: ")
for i in range(len(data.columns)):
    print(end = "\t")
    print(str(i + 1)+". "+data.columns[i])
    
print("Number of elements in each feature of dataset:\n", data.count()) #To get an idea if there are missing values.
print("\n\nCount of NaN values in each column:\n ",data.isna().sum())  #Counting number of missing values in each column

Number of dimensions in dataset :  2
Dimensions of dataset :  (19470, 14)
Number of Features in the dataset:  14
List of features in the dataset: 
	1. Date received
	2. Product
	3. Sub-product
	4. Issue
	5. Sub-issue
	6. Consumer complaint narrative
	7. Company public response
	8. Company
	9. Submitted via
	10. Date sent to company
	11. Company response to consumer
	12. Timely response?
	13. Consumer disputed?
	14. Complaint ID
Number of elements in each feature of dataset:
 Date received                   19470
Product                         19470
Sub-product                     19470
Issue                           19470
Sub-issue                       19470
Consumer complaint narrative    19470
Company public response         19470
Company                         19470
Submitted via                   19470
Date sent to company            19470
Company response to consumer    19470
Timely response?                19470
Consumer disputed?              19470
Complaint ID              

In [111]:
data.columns = [col.strip() for col in data.columns]
data.columns = [col.replace('-',"_") for col in data.columns]
data.columns = [col.replace(' ',"_") for col in data.columns]
data.columns = [col.title() for col in data.columns]

data.rename(columns = {'Consumer_Complaint_Narrative':'Complaint', 'Company_Public_Response':'Response'}, inplace = True)
print(data.columns)

Index(['Date_Received', 'Product', 'Sub_Product', 'Issue', 'Sub_Issue',
       'Complaint', 'Response', 'Company', 'Submitted_Via',
       'Date_Sent_To_Company', 'Company_Response_To_Consumer',
       'Timely_Response?', 'Consumer_Disputed?', 'Complaint_Id'],
      dtype='object')


In [115]:
 #Missing values in each column
print("\n\nCount of NaN values in each column:\n",data.isna().sum()) 



Count of NaN values in each column:
 Date_Received                   0
Product                         0
Sub_Product                     0
Issue                           0
Sub_Issue                       0
Complaint                       0
Response                        0
Company                         0
Submitted_Via                   0
Date_Sent_To_Company            0
Company_Response_To_Consumer    0
Timely_Response?                0
Consumer_Disputed?              0
Complaint_Id                    0
dtype: int64


In [ ]:
# Number of Products

unique_ele = data['Product'].unique()
print("Number of unique elements in Product : ", len(unique_ele))
print("\n\nUnique elements in Product : ", unique_ele)

In [75]:
# Number of Complaints in each Product

temp_series = data.groupby('Product').size()
print(temp_series)

temp_series.sort_values(ascending = False, inplace = True)
print("\nAfter sorting in Descending Order :\n\n",temp_series)

Product
Bank account or service                                                           86205
Checking or savings account                                                      182837
Consumer Loan                                                                     31575
Credit card                                                                       92193
Credit card or prepaid card                                                      206262
Credit reporting                                                                 140429
Credit reporting or other personal consumer reports                               69227
Credit reporting, credit repair services, or other personal consumer reports    2166447
Debt collection                                                                  506859
Debt or credit management                                                            72
Money transfer, virtual currency, or money service                                58810
Money transfers         

In [76]:
# Selecting the Product for Analysis
product_list = ['Credit reporting, credit repair services, or other personal consumer reports' , 'Mortgage' , 'Debt collection', 'Consumer Loan']

print("\n The Product list is : ", product_list)


 The Product list is :  ['Credit reporting, credit repair services, or other personal consumer reports', 'Mortgage', 'Debt collection', 'Consumer Loan']


In [77]:
data_orig = data.copy()

In [78]:
data = data.loc[data['Product'].isin(product_list)]

length  = len(data.Product)
print(length)
data.index = [np.arange(0,length)]

3089109


In [81]:
#Label Encoding to transform alphabetical data into numerical for model fitting

encoder = LabelEncoder()
data['Product_Encoding'] = encoder.fit_transform(data['Product'])
data.head()

,Date_Received,Product,Sub_Product,Issue,Sub_Issue,Complaint,Response,Company,State,Zip_Code,Tags,Consumer_Consent_Provided?,Submitted_Via,Date_Sent_To_Company,Company_Response_To_Consumer,Timely_Response?,Consumer_Disputed?,Complaint_Id,Product_Encoding
0,2023-08-24,"Credit reporting, credit repair services, or o...",Credit reporting,Problem with a credit reporting company's inve...,Was not notified of investigation status or re...,NaN,NaN,Experian Information Solutions Inc.,NJ,07024,NaN,Other,Web,2023-08-24,In progress,Yes,NaN,7452639,1
1,2023-08-23,"Credit reporting, credit repair services, or o...",Credit reporting,Problem with a credit reporting company's inve...,Investigation took more than 30 days,NaN,NaN,"EQUIFAX, INC.",TN,38116,NaN,NaN,Web,2023-08-23,Closed with non-monetary relief,Yes,NaN,7442979,1
2,2023-05-24,"Credit reporting, credit repair services, or o...",Credit reporting,Problem with a credit reporting company's inve...,Their investigation did not fix an error on yo...,"When I reviewed my credit report, I discovered...",Company has responded to the consumer and the ...,Experian Information Solutions Inc.,PA,19145,NaN,Consent provided,Web,2023-05-24,Closed with explanation,Yes,NaN,7020780,1
3,2023-09-20,Debt collection,Other debt,Attempts to collect debt not owed,Debt was paid,NaN,NaN,Kingston Data & Credit International Inc.,GA,30292,NaN,NaN,Phone,2023-09-20,Closed with explanation,Yes,NaN,7581781,2
4,2023-06-13,"Credit reporting, credit repair services, or o...",Credit reporting,Problem with a credit reporting company's inve...,Their investigation did not fix an error on yo...,I submitted a letter to the XXXX Credit Bureau...,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",FL,32221,NaN,Consent provided,Web,2023-06-13,Closed with non-monetary relief,Yes,NaN,7108471,1


In [82]:
# Removing Stopwords and cleaning of Complaints

data['Complaint'] = data['Complaint'].astype(str)
data.dtypes

Date_Received                   object
Product                         object
Sub_Product                     object
Issue                           object
Sub_Issue                       object
Complaint                       object
Response                        object
Company                         object
State                           object
Zip_Code                        object
Tags                            object
Consumer_Consent_Provided?      object
Submitted_Via                   object
Date_Sent_To_Company            object
Company_Response_To_Consumer    object
Timely_Response?                object
Consumer_Disputed?              object
Complaint_Id                     int64
Product_Encoding                 int32
dtype: object

In [84]:
final_data = data[['Complaint' , 'Product' , 'Product_Encoding']]

print(final_data.shape)
final_data.to_csv('Complaint_data.csv' , index='False')

(3089109, 3)


In [89]:
stop_words = set(stopwords.words('english'))

data['Filtered_Text'] = data['Complaint'].apply(lambda x: " ".join([word for word in x.split() if word not in stop_words]))
data.head()

,Date_Received,Product,Sub_Product,Issue,Sub_Issue,Complaint,Response,Company,State,Zip_Code,Tags,Consumer_Consent_Provided?,Submitted_Via,Date_Sent_To_Company,Company_Response_To_Consumer,Timely_Response?,Consumer_Disputed?,Complaint_Id,Product_Encoding,Filtered_Text
0,2023-08-24,"Credit reporting, credit repair services, or o...",Credit reporting,Problem with a credit reporting company's inve...,Was not notified of investigation status or re...,nan,NaN,Experian Information Solutions Inc.,NJ,07024,NaN,Other,Web,2023-08-24,In progress,Yes,NaN,7452639,1,nan
1,2023-08-23,"Credit reporting, credit repair services, or o...",Credit reporting,Problem with a credit reporting company's inve...,Investigation took more than 30 days,nan,NaN,"EQUIFAX, INC.",TN,38116,NaN,NaN,Web,2023-08-23,Closed with non-monetary relief,Yes,NaN,7442979,1,nan
2,2023-05-24,"Credit reporting, credit repair services, or o...",Credit reporting,Problem with a credit reporting company's inve...,Their investigation did not fix an error on yo...,"When I reviewed my credit report, I discovered...",Company has responded to the consumer and the ...,Experian Information Solutions Inc.,PA,19145,NaN,Consent provided,Web,2023-05-24,Closed with explanation,Yes,NaN,7020780,1,"When I reviewed credit report, I discovered in..."
3,2023-09-20,Debt collection,Other debt,Attempts to collect debt not owed,Debt was paid,nan,NaN,Kingston Data & Credit International Inc.,GA,30292,NaN,NaN,Phone,2023-09-20,Closed with explanation,Yes,NaN,7581781,2,nan
4,2023-06-13,"Credit reporting, credit repair services, or o...",Credit reporting,Problem with a credit reporting company's inve...,Their investigation did not fix an error on yo...,I submitted a letter to the XXXX Credit Bureau...,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",FL,32221,NaN,Consent provided,Web,2023-06-13,Closed with non-monetary relief,Yes,NaN,7108471,1,I submitted letter XXXX Credit Bureaus last XX...


In [90]:
new_df = data.groupby('Product').head(7000)
print(new_df['Product'].value_counts())      

data = new_df

Product
Credit reporting, credit repair services, or other personal consumer reports    7000
Debt collection                                                                 7000
Mortgage                                                                        7000
Consumer Loan                                                                   7000
Name: count, dtype: int64


In [91]:
# Dividing the Dataset into Testing and Training set.

target = data['Product_Encoding']
X = data.drop(columns = ['Product_Encoding'])
print("\n List of Input : ",X.columns)

X_train, X_test, target_train, target_test = train_test_split(X, target, test_size = 0.30, random_state = 25)
print(len(X))
print(len(X_train))
print(len(X_test))
print(len(target_train))
print(len(target_test))


 List of Input :  Index(['Date_Received', 'Product', 'Sub_Product', 'Issue', 'Sub_Issue',
       'Complaint', 'Response', 'Company', 'State', 'Zip_Code', 'Tags',
       'Consumer_Consent_Provided?', 'Submitted_Via', 'Date_Sent_To_Company',
       'Company_Response_To_Consumer', 'Timely_Response?',
       'Consumer_Disputed?', 'Complaint_Id', 'Filtered_Text'],
      dtype='object')
28000
19600
8400
19600
8400


In [94]:
vectorizer_1 = TfidfVectorizer(max_features = 5000)
Y = vectorizer_1.fit_transform(X_train['Filtered_Text'])

print("\n Shape : ", Y.shape)
vocab = vectorizer_1.get_feature_names_out() 
print(vocab)


 Shape :  (19600, 5000)
['00' '000' '10' ... 'zero' 'zions' 'zone']


In [95]:
Z = vectorizer_1.transform(X_test['Filtered_Text'].T)
print(" Shape : ", Z.shape)

 Shape :  (8400, 5000)


In [96]:
# Training decision tree classifier

model_type_1 = DecisionTreeClassifier(max_depth = 10, random_state = 20)
model = model_type_1.fit(Y,target_train)
output = model.predict(Z)
confusion_matrix(target_test, output)
accuracy_score(target_test,output)

0.48428571428571426

In [98]:
# Training Random Forest classifier

model_type_2 = RandomForestClassifier(n_estimators = 10, random_state = 20)
model = model_type_2.fit(Y,target_train)
output = model.predict(Z)
print(confusion_matrix(target_test, output))
accuracy_score(target_test,output)

[[1220   25   50  802]
 [  76  584   44 1404]
 [ 192   63  360 1481]
 [ 148    3    4 1944]]


0.48904761904761906

In [99]:
# Training using Naive bayes classifier

model_type_3= GaussianNB()
model = model_type_3.fit(Y.toarray(),target_train)
output = model.predict(Z.toarray())
print(confusion_matrix(target_test, output))
accuracy_score(target_test,output)

[[1163  820   76   38]
 [  61 1936   96   15]
 [ 172 1544  349   31]
 [ 122 1673   25  279]]


0.4436904761904762

In [101]:
New_output = pd.DataFrame(X_test.copy(deep = True))
New_output['Actual_Class'] = target_test
New_output['Predicted_Class'] = output

New_output.index = np.arange(1, New_output.shape[0] + 1)


print(New_output.shape)

print(New_output.tail())

(8400, 21)
     Date_Received                                            Product  \
8396    2013-08-26                                           Mortgage   
8397    2013-08-29                                    Debt collection   
8398    2023-06-28                                    Debt collection   
8399    2023-07-06  Credit reporting, credit repair services, or o...   
8400    2015-07-15                                      Consumer Loan   

                      Sub_Product                                     Issue  \
8396  Conventional fixed mortgage  Loan servicing, payments, escrow account   
8397                I do not know     Cont'd attempts collect debt not owed   
8398             Credit card debt                     Communication tactics   
8399             Credit reporting               Improper use of your report   
8400                 Vehicle loan                Managing the loan or lease   

                                          Sub_Issue  \
8396                

In [103]:
data_2 = New_output.loc[New_output['Actual_Class'] != New_output['Predicted_Class']]
print("\nShape of data whose Actual & predicted class differs = " + str(data_2.shape))


data_2 = data_2.groupby('Actual_Class').head()
print("\n\nTaking 5 samples of each wrongly predicted complaint" + str(data_2.shape))

print(data_2[['Actual_Class', 'Predicted_Class']])


Shape of data whose Actual & predicted class differs = (4027, 21)


Taking 5 samples of each wrongly predicted complaint(20, 21)
     Actual_Class  Predicted_Class
2               2                3
3               2                1
6               0                3
10              1                3
11              1                3
12              0                3
13              2                3
14              1                3
16              2                3
18              2                3
19              1                3
26              1                3
39              3                0
40              0                3
43              0                3
46              0                3
339             3                0
357             3                0
722             3                1
958             3                0
                                             Complaint  Actual_Class  \
2                                                  nan         